FULL IMPLEMENTATION OF VGG-19 NEURAL NETWORK USING TENSORFLOW

In [ ]:
IMPORT DATASET

IMPORT TENSORFLOW V2 - LIMITED

IMPORT NECESSARY LIBRARIES

In [2]:
import os
import cv2
import requests
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.preprocessing import OneHotEncoder
from PIL import Image
from sklearn.model_selection import train_test_split

CONSTRUCT AN ENCODER TO CLASSIFY RESULTS

In [3]:
encoder = OneHotEncoder()
encoder.fit([[0], [1]])

OneHotEncoder()

PRE-PROCESS THE DATASET

In [ ]:
data = []
paths = []
result = []

def pre_process(position, path):
  paths = []
  for x, y, z in os.walk(path):
    for file in z: paths.append(os.path.join(x, file))
              
  for path in paths:
    img = Image.open(path)
    size=(224,224)
    img = img.resize(size)
    img = np.array(img)

    if (img.shape == (224, 224, 3)):
        data.append(np.array(img))
        result.append(encoder.transform([[position]]).toarray())

START THE DATASET PRE-PROCESSING

In [ ]:
pre_process(0, '/content/Dataset/Train/Beans')
pre_process(1, '/content/Dataset/Train/Redonion')

data = np.array(data)
result = np.array(result)
result = result.reshape(data.shape[0], 2)

SPLIT THE DATA FOR BOTH TESTING AND TRAINING

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)

ValueError: ignored

CREATE VGG-19 MODEL USING SEQUENTIAL API

In [ ]:
def createModel(inputShape=(224, 224, 3)):

    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3,3), padding= 'same', activation= 'relu', input_shape= inputShape))
    model.add(Conv2D(64, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    model.add(Conv2D(128, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(128, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    model.add(Conv2D(256, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(256, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(256, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(256, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(6, activation= 'softmax'))

    return model

COMPILE THE NEURAL NETWORK MODEL WITH LEARNING RATE OF 0.001 OF Adamax OPTIMISER

In [ ]:
model = createModel()
model.compile(optimizer= tf.keras.optimizers.Adamax(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

START TRAINING THE MODEL

In [ ]:
history = model.fit(x_train, y_train, epochs = 10, batch_size = 16, verbose = 1,validation_data = (x_test, y_test))

EVALUATE THE LOSS & ACCURACY

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='lower right')
plt.show()

MODEL TESTING AND EVALUATION

In [ ]:
from matplotlib.pyplot import imshow

def names(number):
  return 'Beans' if number==0.0 else 'Red Onion'

In [ ]:
# TEST BEANS
img = Image.open('/content/Dataset/Test/Beans/frame10.jpg')
x = np.array(img.resize((224,224)))
x = x.reshape(1,224,224,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print('Confidence score:', str(res[0][classification]*100) + '%')
print('Result:', names(classification))

In [ ]:
# TEST RED ONION
img = Image.open('/content/Dataset/Test/Redonion/frame100.jpg')
x = np.array(img.resize((224,224)))
x = x.reshape(1,224,224,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print('Confidence score:', str(res[0][classification]*100) + '%')
print('Result:', names(classification))

DICE COEFFICIENT

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, accuracy_score, precision_recall_curve, average_precision_score, plot_precision_recall_curve, f1_score
from matplotlib.pyplot import figure

p_y = model.predict(x_test, verbose=False)
fpr, tpr, th = roc_curve(y_test.argmax(axis=1), p_y.argmax(axis=1), pos_label=1)
fig, c_ax = plt.subplots(1,1, figsize=(9, 9))
c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % ('Classification', auc(fpr, tpr)))
c_ax.plot([0,1], [0,1], color='red', lw=1, linestyle='--')
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')

PRECISION AND RECALL

In [ ]:
p_y = model.predict(x_test, verbose=False)
fig, c_ax = plt.subplots(1,1, figsize=(9, 9))
p, r, th = precision_recall_curve(y_test.argmax(axis=1), p_y.argmax(axis=1), pos_label=1)
aps = average_precision_score(y_test, p_y)
c_ax.plot(r, p, label = '%s (AP Score:%0.2f)'  % ('Classification', aps))
c_ax.plot(r, p, color='navy', lw=2)
c_ax.legend()
c_ax.set_xlabel('Recall')
c_ax.set_ylabel('Precision')

MODEL ARCHITECTURE

In [ ]:
from tensorflow.keras.utils import model_to_dot
from IPython.display import SVG
import pydot
import graphviz

SVG(model_to_dot(model, show_shapes=True, show_layer_names=True, rankdir='TB',expand_nested=False, dpi=60, subgraph=False).create(prog='dot',format='svg'))

SAVE THE MACHINE LEARNING MODEL

In [ ]:
model.save("/content/model_vgg_19.h5")